In [3]:
import cv2
import numpy as np
import time
# A biblioteca picamera2 não é mais necessária

# --- MUDANÇA ESSENCIAL: ARQUIVO DE VÍDEO COMO INPUT ---
VIDEO_FILE_PATH = "video.mp4" 
# Por favor, substitua "video_para_analise.mp4" pelo caminho real do seu arquivo.
# ----------------------------------------------------

# --- PARÂMETROS DE DETECÇÃO (AJUSTE AQUI) ---
LOWER_BLUE = np.array([85, 30, 120])
UPPER_BLUE = np.array([130, 255, 255])

# --- RANGE DO PRETO MAIS TOLERANTE (Ajustado) ---
LOWER_BLACK = np.array([0, 0, 0])
UPPER_BLACK = np.array([180, 255, 70]) 

# Porcentagem mínima da borda que deve ser azul para validar a mancha
BLUE_BORDER_THRESHOLD = 0.60 # 90%
# ----------------------------------------------------

# --- MUDANÇA 1: INICIALIZAÇÃO DO VÍDEO DE ENTRADA ---
print(f"Abrindo o vídeo de entrada: {VIDEO_FILE_PATH}")
cap = cv2.VideoCapture(VIDEO_FILE_PATH)

# Verificar se o arquivo de vídeo foi aberto com sucesso
if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o arquivo de vídeo em {VIDEO_FILE_PATH}")
    # Usar um erro mais explícito para evitar continuação
    raise FileNotFoundError(f"Vídeo não encontrado ou formato inválido: {VIDEO_FILE_PATH}")


Abrindo o vídeo de entrada: video.mp4


In [4]:
# 2. Configuração do Vídeo de Saída, baseada no INPUT
# Obtendo as propriedades do vídeo de entrada
largura = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
altura = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Se o FPS não for detectado ou for zero, usar um padrão (ex: 30)
if fps <= 0:
    fps = 30
    print(f"Aviso: FPS do vídeo de entrada não detectado, usando padrão de {fps}.")
    
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("saida_mascara_validada.mp4", fourcc, fps, (largura, altura))

print(f"Processando vídeo com dimensões: {largura}x{altura} a {fps:.2f} FPS.")

# Variável para guardar o valor final da área (porém, esta variável será a área do ÚLTIMO frame)
area_final_validada = 0
frame_count = 0

# --- KERNEL PARA A LIMPEZA DA MÁSCARA ---
kernel = np.ones((3,3), np.uint8)

Processando vídeo com dimensões: 1920x1080 a 30.00 FPS.


In [5]:
# 3. Loop de Captura e Processamento
start_time = time.time()
while cap.isOpened():
    # cap.read() retorna:
    # ret (True/False): se o frame foi lido com sucesso
    # frame: o próprio frame (imagem BGR)
    ret, frame = cap.read() 

    if not ret:
        # Se ret for False, significa que o vídeo terminou ou houve um erro
        break
    
    frame_count += 1
    
    # Conversão para HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- INÍCIO DA LÓGICA DE VALIDAÇÃO ---
    
    # Criar máscaras para as cores azul e preta
    mask_blue = cv2.inRange(hsv, LOWER_BLUE, UPPER_BLUE)
    
    # 1. Cria a máscara preta "suja" (granulada)
    mask_black_suja = cv2.inRange(hsv, LOWER_BLACK, UPPER_BLACK)

    # 2. Usa o "CLOSE" para preencher os buracos na máscara preta (limpeza)
    mask_black = cv2.morphologyEx(mask_black_suja, cv2.MORPH_CLOSE, kernel)

    # Encontrar todos os contornos pretos (na máscara limpa)
    # OBS: cv2.findContours modifica a máscara de entrada, então usamos uma cópia se necessário,
    # mas aqui estamos usando a `mask_black` que não precisamos mais dela 'pura', então está ok.
    black_contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    valid_contours = []
    total_valid_area_frame = 0

    # Iterar sobre cada contorno preto encontrado
    for cnt in black_contours:
        # Criar uma máscara temporária apenas com o contorno atual
        mask_temp = np.zeros_like(mask_black)
        cv2.drawContours(mask_temp, [cnt], -1, 255, -1)

        # Dilatar a máscara para criar uma "borda" ao redor da mancha
        kernel_borda = np.ones((5,5), np.uint8) 
        dilated_mask = cv2.dilate(mask_temp, kernel_borda)
        
        # Subtrair a máscara original da dilatada para obter apenas a borda
        border = cv2.subtract(dilated_mask, mask_temp)

        # --- Produção: cálculo de proporção usando máscara azul limpa/dilatada ---
        # Limpar a máscara azul para reduzir pequenos buracos (ajuda em anti-aliasing)
        mask_blue_clean = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))

        # Dilatar ligeiramente a máscara azul para compensar anti-aliasing e pequenos gaps
        mask_blue_dilated = cv2.dilate(mask_blue_clean, np.ones((3,3), np.uint8))

        # Contagem de pixels da borda
        total_border_pixels = int(np.sum(border > 0))
        if total_border_pixels == 0:
            continue  # sem borda relevante, passa para próximo contorno

        # Usa a máscara dilatada para maior tolerância ao comparar com a borda
        border_blue_pixels = cv2.bitwise_and(mask_blue_dilated, mask_blue_dilated, mask=border)
        total_blue_border_pixels = int(np.sum(border_blue_pixels > 0))

        # Calcular a proporção de azul na borda
        blue_ratio = total_blue_border_pixels / total_border_pixels

        # Se a proporção for maior que o nosso limite, o contorno é válido
        if blue_ratio > BLUE_BORDER_THRESHOLD:
            valid_contours.append(cnt)
            total_valid_area_frame += int(cv2.contourArea(cnt))
    
    # Atualiza a área final com a área total do frame atual
    area_final_validada = total_valid_area_frame

    # --- FIM DA LÓGICA DE VALIDAÇÃO ---

    # 4. Salvar o frame da MÁSCARA VALIDADA no vídeo de saída
    # Criamos um frame preto (baseado nas dimensões da mask_black limpa)
    final_mask = np.zeros_like(mask_black)
    if valid_contours:
        # Desenha APENAS as manchas validadas (que agora são sólidas)
        cv2.drawContours(final_mask, valid_contours, -1, 255, -1)

    # Converter a máscara final para BGR para salvar no vídeo
    mascara_colorida = cv2.cvtColor(final_mask, cv2.COLOR_GRAY2BGR)
    out.write(mascara_colorida)

# 5. Finalização
end_time = time.time()
elapsed_time = end_time - start_time
print("\nProcesso concluído!")
print(f"Frames processados: {frame_count}")
print(f"Tempo total de processamento: {elapsed_time:.2f} segundos")

# --- MUDANÇA 2: FINALIZAÇÃO DO CAPTURE E WRITER ---
# Fechar janelas caso tenham sido abertas durante debug
cv2.destroyAllWindows()
cap.release() # Fecha o arquivo de vídeo de entrada
out.release() # Fecha o arquivo de vídeo de saída

# 6. Exibir os resultados finais
print(f"Vídeo da máscara validada salvo em: saida_mascara_validada.mp4")
print(f"Área total final das manchas validadas no ÚLTIMO frame: {int(area_final_validada)} pixels")



Processo concluído!
Frames processados: 3216
Tempo total de processamento: 123.70 segundos
Vídeo da máscara validada salvo em: saida_mascara_validada.mp4
Área total final das manchas validadas no ÚLTIMO frame: 7466 pixels
